In [1]:
# This cell contains the parameters. It should have the tag "parameters"
SRCDIR = "."
DATA_RANGE = 255
NOISE_LEVEL = 10
NOISE_LEVEL_SIGNAL = .10
noise_type_structured = True
noise_type_unstructured = True
noise_type_structured_signal = True
noise_type_unstructured_signal = True
add_noise = True
SIZE = 3
THRES = 200
kernel = "bilateral"
M = 10

In [2]:
import imageio
import tarfile
import numpy as np
import seaborn as sns
import os
import skimage
import skimage.io as io
from IPython.display import display, Image
import scipy
from scipy import signal
from scipy.signal import fftconvolve
from zipfile import ZipFile

In [3]:
# Input path for the image
dir_path = SRCDIR + "/input_0.zip"

with ZipFile(dir_path, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')
    zip.infolist()
    list_img_path = [info.filename for info in zip.infolist() if info.filename[-3:]=="png" or info.filename[-3:]=="jpg"]

File Name                                             Modified             Size
00000.jpg                                      2016-08-03 11:32:22       115360
00001.jpg                                      2016-08-03 11:32:22       108764
00002.jpg                                      2016-08-03 11:32:22       111677
00003.jpg                                      2016-08-03 11:32:22       105974
00004.jpg                                      2016-08-03 11:32:22       106208
00005.jpg                                      2016-08-03 11:32:22       102985
00006.jpg                                      2016-08-03 11:32:22       101478
00007.jpg                                      2016-08-03 11:32:22       113109
00008.jpg                                      2016-08-03 11:32:22       116155
00009.jpg                                      2016-08-03 11:32:22       116499
00010.jpg                                      2016-08-03 11:32:22       117267
00011.jpg                               

In [4]:
nb_images = len(list_img_path)

In [6]:
file = open("algo_info.txt","w")
file.write("num_outputs="+ str(nb_images))
file.close()

In [ ]:
#list_img_path = ["input_" + str(i) + ".png"]
#list_img_path = [SRCDIR + '/' + '0' * (5-len(str(i))) + str(i) + ".jpg" for i in range(50)]

In [ ]:
#print("imageio", imageio.__version__)
#print("numpy", np.__version__)
#print("skimage", skimage.__version__)
#print("scipy", scipy.__version__)

In [ ]:
def l1_norm(x):
    return (np.abs(x)).sum()

def roughness_index(img):
    h1 = np.array([[1, -1]])
    h2 = h1.T
    img = np.squeeze(img)
    if len(img.shape) == 2:
        res = l1_norm(signal.fftconvolve(img, h1, mode='full'))
        res += l1_norm(signal.fftconvolve(h2, img, mode='full'))
        res = res / l1_norm(img)
        return res
    elif len(img.shape) == 3:
        res = 0
        for i in range(3):
            res_ = l1_norm(signal.fftconvolve(img[i, :, :], h1, mode='full'))
            res_ += l1_norm(signal.fftconvolve(h2, img[i, :, :], mode='full'))
            res_ = res_ / l1_norm(img)
            res += res_ / 3
        return res
    
def psnr(img1, img2, max_value=1):
    mse = ((img1 - img2) ** 2).mean()
    return 10 * np.log10(max_value ** 2 / mse)

def return_splatial_mean(img, size, C):
    ones = np.ones_like(img)
    spatial_mean = np.zeros_like(img)
    kernel = np.ones((size, size))

    for chanel_index in range(C):
        convol_image = fftconvolve(img[chanel_index, :, :], kernel, mode="same")
        ns = fftconvolve(ones[chanel_index, :, :], kernel, mode="same")
        spatial_mean[chanel_index, :, :] = convol_image / ns

    return spatial_mean

In [ ]:
def return_gaussian_noise(mean, noise_level, H, W, C=1):
    """
    :param C: number of channels of the image we want to add noise
    :param W: width of the image we want to add noise
    :param H: height of the image we want to add noise
    :param mean: mean of the noise we want to add
    :param noise_level: standard deviation of the noise we want to add
    :return:
    """
    noise = np.random.normal(size=(H, W, C), loc=mean, scale=noise_level)
    return noise


def return_colum_row_noise(mean, noise_level, H, W, C=1, add=True):
    """
    :param C: number of channels of the image we want to add noise
    :param W: width of the image we want to add noise
    :param H: height of the image we want to add noise
    :param mean: mean of the noise we want to add
    :param noise_level: standard deviation of the noise we want to add
    :return:
    """
    noise = np.zeros((H, W, C))
    for i in range(C):
        noise_column = np.random.normal(size=(H, 1), loc=mean, scale=noise_level) @ np.ones((1, W))
        noise_row = np.ones((H, 1)) @ np.random.normal(size=(1, W), loc=mean, scale=noise_level)
        if add:
            noise[:, :, i] = noise_row + noise_column
        else:
            noise[:, :, i] = noise_row * noise_column
    return noise

In [ ]:
img = imageio.imread(list_img_path[0])
img_shape = img.shape
if len(img_shape) == 3:
    H, W, C = img_shape
elif len(img_shape) == 2:
    H, W = img_shape
    C = 1
data_type = img.dtype

In [ ]:
imgs = []
for i in range(nb_images):
    img = imageio.imread(list_img_path[i])
    imgs.append(img)

In [ ]:
noise = np.zeros((H, W, C))
noise_signal = np.ones((H, W, C))

unstructured_noise_offset = return_gaussian_noise(0, NOISE_LEVEL, H=H, W=W, C=C)
structured_noise_offset = return_colum_row_noise(0, NOISE_LEVEL, H=H, W=W, C=C)

unstructured_noise_signal = return_gaussian_noise(1, NOISE_LEVEL_SIGNAL, H=H, W=W, C=C)
structured_noise_signal = return_colum_row_noise(1, NOISE_LEVEL_SIGNAL, H=H, W=W, C=C, add=False)

if noise_type_structured:
    noise += structured_noise_offset
if noise_type_unstructured:
    noise += unstructured_noise_offset
if noise_type_structured_signal:
    noise_signal *= structured_noise_signal
if noise_type_unstructured_signal:
    noise_signal *= unstructured_noise_signal

In [ ]:
# gain correction
G = 1.0
# offset correction
O = np.zeros((H, W, C))

In [ ]:
loss_mse_hist = []
psnr_hist = []
psnr_noisy = []
roughness_denoised = []
roughness_noisy = []
roughness_clean = []

In [ ]:
for index_image in range(nb_images):
    img = imgs[index_image]
    io.imsave("input_" + str(index_image) + '.png', img.astype(data_type))
    img = np.array(img, dtype=float)
    if C > 1:
        img = img
    elif C == 1:
        img = img[None, :, :]
    if add_noise:
        noisy_img = img * noise_signal + noise
        noisy_img = noisy_img.round()
        noisy_img[noisy_img < 0] = 0
        noisy_img[noisy_img > DATA_RANGE] = DATA_RANGE
        io.imsave("noisy_" + str(index_image) + '.png', noisy_img.astype(data_type))
    else:
        noisy_img = img

    if add_noise:
        img = imgs[index_image]
        img = img

    # denoise image
    y = (G * noisy_img - O)
    y = y.round()
    y[y < 0] = 0
    y[y > DATA_RANGE] = DATA_RANGE

    if add_noise:
        psnr_hist.append(psnr(y, img, max_value=DATA_RANGE))
        psnr_noisy.append(psnr(noisy_img, img, max_value=DATA_RANGE))
        roughness_clean.append(roughness_index(img))


    roughness_noisy.append(roughness_index(noisy_img))
    roughness_denoised.append(roughness_index(y))
    
    if kernel == None or kernel == "None" or kernel == "none":
        Y_HSF = noisy_img
    elif kernel == "mean":
        y_LSF = return_splatial_mean(noisy_img, SIZE, C)
        y_HSF = noisy_img - y_LSF
        Y_HSF = y_HSF.copy()
        Y_HSF[np.abs(y_HSF) > THRES] = 0
    elif kernel == "bilateral":
        y_LSF = skimage.restoration.denoise_bilateral(noisy_img, SIZE, 75, 75, channel_axis=2)
        Y_HSF = noisy_img - y_LSF

    # TODO update gain and offset correction parameters
    G = 1.0
    O = 1 / M * Y_HSF + (1 - 1 / M) * O

    # save denoised image
    y = y.astype(data_type)
    io.imsave("output_" + str(index_image) + '.png', y)

::: {.panel-tabset}

### Input image

In [ ]:
for i in range(nb_images):
    print("roughness", roughness_clean[i])
    print("PSNR", 100)
    display(Image("input_" + str(i) + ".png"))

### Noisy image

In [ ]:
for i in range(nb_images):
    print("roughness", roughness_noisy[i])
    print("PSNR", psnr_noisy[i])
    display(Image("noisy_" + str(i) + ".png"))

### Output image

In [ ]:
for i in range(nb_images):
    print("roughness", roughness_denoised[i])
    print("PSNR", psnr_hist[i])
    display(Image("output_" + str(i) + ".png"))

:::